In [1]:
import numpy as np
import matplotlib.pyplot as plt
import leidenalg as la
import igraph as ig
import csv
from read_roi import read_roi_file, read_roi_zip
from glob import glob
import os

In [10]:
class temporal_network:
    def __init__(self, size, length, data, **kwargs):
        
        if length < 2: return('Object should be a multilayer network with at least 2 layers')
        if size < 3: return('Layers must have at least 3 nodes')
        
        self.size = size # number of nodes in every layer
        self.length = length # number of layers
        self.nodes = [i for i in range(self.size)]
        
        #### data: you can go back and forth between supra adjacency and list adjacency but not to edge list
        
        ##         if supra__adjacency: creates the list adjacency matrix
        ##
        ##                     --- additional arguments ---
        ##                       - supra_adjacency: supra adjacency matrix of shape (size*time x size*time)
        
        
        ##
        ##         if edge__list: creates undirected binary multilayer network from the egde triplets
        ##                      given of the form (i,j,t). supra_adjacency and list_adjacency matrices 
        ##                      are automatically created. 
        ##
        ##                     --- additional arguments ---
        ##                       - edge_list: list of triplets e.g. [(0,2,1),(2,1,1),(0,1,2),(0,2,2)]
        ##                       - omega: interlayer coupling strength, can be a float only for now
        ##                       - kind: if temporal, only adjacent layers gets connected with strength 'omega'
        ##                               if multilayer, all layers get connected w/ each other w/ strength 'omega'
        
        
        ##         if list__adjacency: creates the supra adjacency matrix
        ##              
        ##                     --- additional arguments ---
        ##                       - list_adjacency: list of length 'length' that contains individual adjacency
        ##                                         matrices of each layer
        ##                       - omega: interlayer coupling strength, can be a float only for now
        ##                       - kind : if temporal, only adjacent layers gets connected w/ strength 'omega'
        ##                                if multilayer, all layers get connected w/ each other w/ strength 'omega'
        
                    
        if  data == 'supra__adjacency':
            self.supra_adjacency = kwargs['supra_adjacency']
            list_adjacency = np.zeros((self.length, self.size, self.size))
            
            for i in range(self.length-1):
                list_adjacency[i][i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size] = self.supra_adjacency[i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size]
            
            self.list_adjacency = list_adjacency
        
        elif data == 'edge__list':
            self.edgelist = kwargs['edge_list']
            supra_adjacency = np.zeros((self.size*self.length,self.size*self.length))
            list_adjacency = np.zeros((self.length, self.size, self.size))
            
            for k,e in enumerate(self.edge_list):
                i,j,t = e[0],e[1],e[2]
                supra_adjacency[self.size*(t-1)+i][self.size*(t-1)+j] = 1
                supra_adjacency[self.size*(t-1)+j][self.size*(t-1)+i] = 1
                list_adjacency[t-1][i][j] = 1
                list_adjacency[t-1][j][i] = 1
        
            ##filling off-diagonal blocks
            if kwargs['kind'] == 'temporal':
                for n in range(self.size*(self.length-1)):
                    supra_adjacency[n][n+self.size] = kwargs['omega']
                    supra_adjacency[n+self.size][n] = kwargs['omega']
                
            elif kwargs['kind'] == 'multilayer':
                i = 0
                while self.length-i != 0:
                    i = i+1
                    for n in range(self.size*(self.length-i)):
                        supra_adjacency[n][n+i*self.size] = kwargs['omega']
                        supra_adjacency[n+i*self.size][n] = kwargs['omega']
            
            self.supra_adjacency = supra_adjacency
            self.list_adjacency = list_adjacency
            
        elif data == 'list__adjacency':
            self.list_adjacency = kwargs['list_adjacency']
            supra_adjacency = np.zeros((self.size*self.length,self.size*self.length))
            
            for i in range(self.length):
                supra_adjacency[i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size] = self.list_adjacency[i]
            
            ##filling off-diagonal blocks
            if kwargs['kind'] == 'temporal':
                for n in range(self.size*(self.length-1)):
                    supra_adjacency[n][n+self.size] = kwargs['omega']
                    supra_adjacency[n+self.size][n] = kwargs['omega']
                
            elif kwargs['kind'] == 'multilayer':
                i = 0
                while self.length-i != 0:
                    i = i+1
                    for n in range(self.size*(self.length-i)):
                        supra_adjacency[n][n+i*self.size] = kwargs['omega']
                        supra_adjacency[n+i*self.size][n] = kwargs['omega']
            
            self.supra_adjacency = supra_adjacency
            
    def aggragate(self, normalized = True):
        t = self.time 
        n = self.size
        aggragated = np.zeros((n,n))
        
        for i,c in enumerate(self.multi_array[:]):
            aggragated = aggragated + c
            
        if normalized: return (aggragated/t)
        else: return (aggragated)
            
    def modularity_matrix(self, omega, gamma):
        N = self.size
        T = self.time
        B = np.zeros((N*T,N*T))
        two_mu = 0
        for i in range(T):
            k = np.sum(self.multi_array[i],0)
            two_m = np.sum(k,0)
            two_mu = two_mu + two_m
            B[i*N:(i+1)*N,i*N:(i+1)*N] = self.multi_array[i] - (gamma * k.T*k)/(two_m)
        two_mu = two_mu + 2*omega*N*(T-1)
        
        for p in range(N*(T-1)):
            B[p][p+N] = omega 
            B[p+N][p] = omega
            
        return(B)
    def edgelist2edges(self):
        T = self.time
        all_edges = [[] for i in range(T)]
        for k,e in enumerate(np.sort(self.edgelist, axis=-1)):
            i,j,t = e[0], e[1], e[2]
            pair = (i,j)
            all_edges[t-1].append(pair)
        return (all_edges)
    
    
    def create_igraph(self, omega):
        T = self.time
        N = self.size
        G = []
        for i in range(T):
            G.append(ig.Graph())
            G[i].add_vertices(N)
            G[i].add_edges(self.edgelist2edges()[i])
            G[i].vs['id'] = list(range(N))
            G[i].vs['node_size'] = 0
            
        G_coupling = ig.Graph.Formula('1 -- 2 -- 3 -- 4');
        G_coupling.es['weight'] = omega; # Interslice coupling strength
        G_coupling.vs['slice'] = [G[0], G[1], G[2], G[3]]
        
        layers, interslice_layer, G_full = la.time_slices_to_layers([G[0],G[1],G[2],G[3]],interslice_weight=0.1);
        partitions = [la.CPMVertexPartition(H, node_sizes='node_size',
                                            weights='weight', resolution_parameter=omega) for H in layers];
        interslice_partition = la.CPMVertexPartition(interslice_layer, resolution_parameter=omega)
                                                     
        return(partitions, interslice_partition)
    
    def leiden(self):
        optimiser = la.Optimiser()
        diff = optimiser.optimise_partition_multiplex(self.create_igraph_try(omega=0)[0] + [self.create_igraph_try(omega=0)[1]])

        return(diff)

In [3]:
def normalized_cross_corr(x,y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    x_cov_std = np.nanmax(np.sqrt(np.correlate(x - x_mean, x - x_mean, 'full')))
    y_cov_std = np.nanmax(np.sqrt(np.correlate(y - y_mean, y - y_mean, 'full')))

    normalization = x_cov_std * y_cov_std
        

    unnormalized_correlation = np.correlate(x - x_mean, y - y_mean, 'full')
    
    corr_array = unnormalized_correlation/normalization

    return(corr_array)

def max_norm_cross_corr(x1, x2):
    
    correlation= normalized_cross_corr(x1, x2)
    
    lag = abs(correlation).argmax() - len(x1)+1
    
    max_corr = max(abs(correlation))
    
    return(max_corr, lag)

def cross_correlation_matrix(data):
    #input: n x t matrix where n is the number of rois and t is the duration of the time series
    #return: n x n cross correlation matrix and lag matrix
    n, t = data.shape
    X = np.zeros((n,n))
    lag = np.zeros((n,n))
    
    for i in range(n-1):
        for j in range(i+1,n):
            X[i][j],lag[i][j] = max_norm_cross_corr(data[i,:],data[j,:])
    X[np.isnan(X)] = 0
    lag[np.isnan(lag)] = 0
    
    X = X + X.T
    lag = lag + lag.T
    return(X,lag)

In [4]:
def read_csv(path, output, subject, roi, subject_roi):
    trace=open( path + output + subject + "_trace.csv", "r")
    spike=open( path + output + subject + "_spikes_complexity.csv", "r")
    reader_trace = csv.reader(trace)
    reader_spike = csv.reader(spike)
    n = read_roi(path, roi, subject_roi)
    traces = np.zeros((n,8000)) # roi x time
    spikes = np.zeros((n,8000)) # roi x time
    #row_count = sum(1 for row in reader)
    
    for i,line in enumerate(reader_trace):
        for j in range(len(line)):
            traces[i][j]=line[j]
    for i,line in enumerate(reader_spike):
        for j in range(len(line)):
            spikes[i][j]=line[j]
    return(traces, spikes)
            
def read_roi(path, roi, subject_roi):
    roi = read_roi_zip(glob(path+roi+subject_roi +'.zip')[0])
    n = len(roi)
    for i, R in enumerate(roi):
        x = roi[R]['x']
        y = roi[R]['y']
    return(n)

def bin_time_series(array, binsize):
    binned_spikes = []
    for i in range(len(array)):
        binned_spikes.append(array[i].reshape(binsize,int(8000/binsize)))
    return(np.array(binned_spikes))

In [5]:
path = '/Users/bengieru/MLN/data/' ## base path
output = 'Johan_Clean_Traces_Features_and_Spikes' #spikes and traces file
roi = 'sarah_ROI' #roi file
subject_roi = '/mouse_1_session_1_baseline' #subject
subject = '/m_1_session_1_baseline'# subject

In [6]:
time = 500 ## binning the time into chunks of
n = read_roi(path, roi, subject_roi) ## number of rois
t = int(8000/time) ## number of layers
traces, spikes = read_csv(path, output, subject, roi, subject_roi) #read the networks
binned_spikes = bin_time_series(spikes, time) # bin the spikes into fixed length

In [7]:
##create cross-correlation matrices that are the adjacency matrices of the network at each layer
adjacency_matrices=[]
for i in range(binned_spikes.shape[2]):
    adjacency_matrices.append(cross_correlation_matrix(binned_spikes[:,:,i])[0]) #

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sqrt
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [8]:
TN = temporal_network(n, t, data = 'list__adjacency', list_adjacency = adjacency_matrices, omega = 1, kind= 'temporal')

In [9]:
TN.supra_adjacency

array([[0.        , 0.27184405, 0.36955297, ..., 0.        , 0.        ,
        0.        ],
       [0.27184405, 0.        , 0.73283274, ..., 0.        , 0.        ,
        0.        ],
       [0.36955297, 0.73283274, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.57997323],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.57997323, 0.        ,
        0.        ]])